In [1]:
import PySimpleGUI as sg

sg.theme("DarkBlue7")
color1 = sg.theme_input_background_color()
color = sg.theme_button_color_background()

In [2]:
def make_view_system_tab(file_id):
    """
    Creates the view system tab for a specific file.

    This function creates the view system tab for a specific file identified by the
    `file_id`. It initializes and sets up the layout for the view system tab.

    Parameters:
        file_id (str): The identifier for the specific file.

    Returns:
        sg.Tab: The view system tab.

    Example usage:
        >>> view_system_tab = make_view_system_tab('file123')
    """

    layout = [
        [sg.Canvas(key=f'controls_cv_view_{file_id}'), sg.Push(),
         sg.FilesBrowse('Load Dump Files...',
                        enable_events=True, key=f'file_view_{file_id}',
                        file_types=(('LAMMPS dump Files', '*.dump'),),
                        target=(0, 2)),
         sg.B('Clear Dump Data',
             key=f'clear_dump_data_{file_id}', disabled=True),
         sg.B('Plot', key=f'plot_view_{file_id}')
         ],
        [sg.T('Run:'), sg.Combo([], key=f'combo_n_view_{file_id}', expand_x=True),
            sg.T('X:'), sg.Combo(
                [], key=f'combo_x_view_{file_id}', expand_x=True),
            sg.T('Y:'), sg.Combo(
                [], key=f'combo_y_view_{file_id}', expand_x=True)],
        [sg.Canvas(key=f'fig_cv_view_{file_id}',
                       # it's important that you set this size
                       size=(600 * 2, 600), expand_x=True, expand_y=True)]
    ]
    
    return sg.Tab('View System', layout=layout, expand_x=True, expand_y=True, key=f'view_system_{file_id}')

In [3]:
def make_rdf_tab(file_id):
    """
    Creates the radial distribution function (RDF) tab for a specific file.

    This function creates the RDF tab for a specific file identified by the
    `file_id`. It initializes and sets up the layout for the RDF tab.

    Parameters:
        file_id (str): The identifier for the specific file.

    Returns:
        sg.Tab: The RDF tab.
    """

    layout = [
        [sg.Canvas(key=f'controls_cv_rdf_{file_id}'), sg.Push(),
         sg.FilesBrowse('Load RDF Files...',
                        enable_events=True, key=f'file_rdf_{file_id}',
                        file_types=(('LAMMPS RDF Files', '*.rdf'),),
                        target=(0, 2)),
         sg.B('Clear RDF Data', key=f'clear_rdf_data_{file_id}', disabled=True),
         sg.B('Plot', key=f'plot_rdf_{file_id}'),
         sg.Input(key=f'rdf_xy_{file_id}', disabled=True, use_readonly_for_disable=True,
                  disabled_readonly_background_color=color1, size=(20, 0))
         ],
        [sg.T('Run:'), sg.Combo([], key=f'combo_n_rdf_{file_id}', expand_x=True),
            sg.T('X:'), sg.Combo(
                [], key=f'combo_x_rdf_{file_id}', expand_x=True),
            sg.T('Y:'), sg.Combo(
                [], key=f'combo_y_rdf_{file_id}', expand_x=True),
            sg.T('RDF Col:'), sg.Combo(
                [], key=f'combo_col_rdf_{file_id}', expand_x=True, disabled=True)
         ],
        [sg.Canvas(key=f'fig_cv_rdf_{file_id}',
                       # it's important that you set this size
                       size=(600 * 2, 600), expand_x=True, expand_y=True
                   )]
    ]
    return sg.Tab('RDF', layout=layout, expand_x=True, expand_y=True, key=f'rdf_{file_id}')

In [4]:
def make_linear_fit_tab(file_id):
    """
    Creates the linear fit tab for a specific file.

    This function creates the linear fit tab for a specific file identified by the
    `file_id`. It initializes and sets up the layout for the linear fit tab.

    Parameters:
        file_id (str): The identifier for the specific file.

    Returns:
        sg.Tab: The linear fit tab.
    """

    fit_headings = ['Line', 'Slope', 'Intercept', 'Boundry 1', 'Boundry 2']
    table = sg.Table(values=[], headings=fit_headings,
                     enable_events=True,
                     enable_click_events=True,
                     expand_x=True,
                     expand_y=True,
                     num_rows=4,
                     select_mode='extended',
                     key=f'table_fit_{file_id}')

    layout = [
        [sg.Canvas(key=f'controls_cv_fit_{file_id}'),
         sg.Push(),
         sg.B('Plot', key=f'plot_fit_{file_id}'),
         sg.B('Intersection point', key=f'intersection_{file_id}'),
         sg.Input(key=f'intersection_value_{file_id}', disabled=True, use_readonly_for_disable=True,
                  disabled_readonly_background_color=color, size=(20, 0))
         ],
        [sg.T('Run:'), sg.Combo([], key=f'combo_n_fit_{file_id}', expand_x=True),
            sg.T('X:'), sg.Combo(
                [], key=f'combo_x_fit_{file_id}', expand_x=True),
            sg.T('Y:'), sg.Combo(
                [], key=f'combo_y_fit_{file_id}', expand_x=True)],
        [sg.Canvas(key=f'fig_cv_fit_{file_id}',
                       # it's important that you set this size
                       size=(600 * 2, 600), expand_x=True, expand_y=True
                   )],
        [table]
    ]
    
    return sg.Tab('Linear Fit', layout=layout, expand_x=True, expand_y=True, key=f'linear_fit_{file_id}')

In [ ]:
def make_overview_tab(file_id):
    """
    Creates the overview tab for a specific file.

    This function creates the overview tab for a specific file identified by the
    `file_id`. It initializes and sets up the layout for the overview tab.

    Parameters:
        file_id (str): The identifier for the specific file.

    Returns:
        sg.Tab: The overview tab.
    """

    frame_layout = []
    for i in range(1, 5):
        frame_layout.append([[sg.T('Run:'), sg.Combo([], key=f'combo_n{i}_{file_id}', expand_x=True),
                             sg.T('X:'), sg.Combo(
                                 [], key=f'combo_x{i}_{file_id}', expand_x=True),
                             sg.T('Y:'), sg.Combo([], key=f'combo_y{i}_{file_id}', expand_x=True)]])

    layout = [
        [sg.Canvas(key=f'controls_cv_overview_{file_id}'),
         sg.Push(),
         sg.B('Plot', key=f'plot_overview_{file_id}'), sg.B(
             'Close Tab', key=f'close_tab_{file_id}')
         ],
        [sg.Frame('', frame_layout[0], expand_x=True),
         sg.Frame('', frame_layout[1], expand_x=True)],
        [sg.Frame('', frame_layout[2], expand_x=True),
         sg.Frame('', frame_layout[3], expand_x=True)],
        [sg.Canvas(key=f'fig_cv_overview_{file_id}',
                   size=(600 * 2, 600), expand_x=True, expand_y=True
                   )]
    ]
    return sg.Tab('Overview', layout=layout, expand_x=True, expand_y=True, key=f'overview_{file_id}')

In [6]:
# Method used to create new tabs in the main window for each loaded file.
def make_new_tab(tab_name, file_id):
    """
    Creates a new tab in the application window for a specific file.

    This function creates a new tab in the application window for a specific file
    identified by the `file_id`. It initializes and sets up the layout for the tab,
    including sub-tabs such as `overview`, `linear_fit`, `rdf`, and `view_system`.

    Parameters:
        tab_name (str): The name of the tab.
        file_id (str): The identifier for the specific file.

    Returns:
        sg.Tab: The newly created tab.
    """
    
    overview = make_overview_tab(file_id)
    linear_fit = make_linear_fit_tab(file_id)
    rdf = make_rdf_tab(file_id)
    view_system = make_view_system_tab(file_id)
    layout = [
        [sg.TabGroup([[overview, linear_fit, rdf, view_system]], key=f'tab_group_{file_id}', expand_x=True, expand_y=True)]]

    return sg.Tab(tab_name, layout=layout, expand_x=True, expand_y=True, key=file_id)

In [7]:
# Method used only once to create the main window of the program.
def make_window():
    """
    Creates the main application window.

    This function initializes and sets up the main application window. It creates
    the necessary elements, such as the title, layout, and
    returns the window object.

    Returns:
        sg.Window: The main application window.
    """

    layout = [
        [sg.FilesBrowse('Load Log Files...',
                        enable_events=True, key='file_log', target= (0,0),
                        file_types=(('LAMMPS Log Files', 'log.*'),
                                    ('LAMMPS Log Files', '*.lammps'),
                                    ('LAMMPS Log Files', '*.log'),
                                    ('LAMMPS Log Files', '*.out')))],
        [sg.TabGroup([[]], key='tab_group', expand_x=True, expand_y=True)]

    ]
    return sg.Window('MDGlass viewer', layout, resizable=True, finalize=True)